# Date grouping revisited

This notebook describes how the date grouping works in bps_to_omop.

## 1.1 Remove overlap

### General rationale

Imagine this as a visit in a timeline

```python
-(- - -)- - - - - - - -
 |     |
 |     |-> End of the visit
 |
 |-> Start of the visit
```
We want to find the **main visits**:

- A main visit contains other visits. 
- Main visit can be next to each other, ie: The end of the 1st can be the start of the 2nd
- Main visits can not overlap each other, ie: The end of the 1st can not be after the start of the 2nd
- Only main visits can populate the VISIT_OCCURRENCE table

If we sort the visits by person_id (asc), start_date (asc), end_date (desc) y type_concet (asc), we will have something like this for each person:
```python
-(- - -)- - - - - - - - 
-(- -)- - - - - - - - -
-(-)- - - - - - - - - -
- -(- - -)- - - - - - -
- -(- -)- - - - - - - -
- -(-)- - - - - - - - - 
- - -(- - -)- - - - - -
- - -(- -)- - - - - - -
- - -(-)- - - - - - - -
- - - - - -(- - -)- - - 
- - - - - -(- -)- - - -
- - - - - -(-)- - - - -
```

If we compare each visit with the **FIRST ONE**, there are different cases here:
1. **COMPLETELY CONTAINED VISITS**: Contained visits are completely contained in the first visit we are considering (This includes consecutive single day visits)
2. **PARTIALLY CONTAINED VISITS**:The start of the visit happens after the start of the 1st, but end of the happens after the end of the 1st
    - Starts afterwards, but extends further into the future.
3. **NOT CONTAINED VISITS**: The start of the visit is after the 1st. It is a "new" main visit.

```python
-(- - -)- - - - - - - - # This is a main visit
-(- -)- - - - - - - - - 1. contained
-(-)- - - - - - - - - - 1. contained
- -(- - -)- - - - - - - 2. partial
- -(- -)- - - - - - - - 1. contained
- -(-)- - - - - - - - - 1. contained
- - -(- - -)- - - - - - 2. partial
- - -(- -)- - - - - - - 2. partial
- - -(-)- - - - - - - - 1. contained
- - - - - -(- - -)- - - # This is the next main visit
- - - - - -(- -)- - - - 1. contained
- - - - - -(-)- - - - - 1. contained
```

Completely contained visits are the easy ones. We can link them to the main visit and remove them.

Partially contained visits are problematic. These will force us to extend our initial visit further into the future, essentially creating a new record.




### Pseudocode explanation

First we join together all files to be used. This essentialy is the VISIT_DETAIL table.

With this initial table we do the following:

1. Build the frame for the VISIT_DETAIL table (`build_visit_detail()`).
   1. Sort by `person_id`(asc), `start_date` (asc), `end_date` (desc) and `type_concept` (asc).
      * This way, we make sure that, for each `start_date`, the first column will have the latest `end_date`. This latest `end_date` is the visit that might contain other visits with the same `start_date`.
      * The `type_concept` column should be transformed first to a category datatype with a predefined order. This way the sorting will take that into account as well.
   2. Generate the field `visit_detail_id` with every visit.
   3. Rename columns to fit the VISIT_DETAIL structure.
   
With this, we have the VISIT_DETAIL table. From here we can build VISIT_OCCURRENCE on top of VISIT_DETAIL.

2. Extend the VISIT_DETAIL table to prepare for VISIT_OCCURRENCE generation (`build_visit_detail_extended()`):
   1. Generate placeholders for future VISIT_OCCURRENCE columns and for temporary columns.
      1. Group by each patient, generating:
        - **visit_start_datetime**: Initially the first **visit_detail_start_datetime**. It will be the **visit_start_datetime** VISIT_OCCCURRENCE column.
        - **visit_end_datetime**: Initially the first **visit_detail_end_datetime**. It will be the **visit_start_datetime** VISIT_OCCCURRENCE column. 
          - If visits needs to be joined together, this record will have the latest joined visit.
        - **visit_detail_id_original**: Initially the first **visit_detail_id**. It will be the **visit_detail_id** VISIT_OCCCURRENCE column.
        - **main_visit**: Initially will have a constant *"Unknown"* value for every record. It will be updated to identify main visits (Changed to "Yes") and the rest (Changed to "No").
      2. Join with VISIT_DETAIL to get the first row (earliest start_date, latest end_date) for each patient.
   2. Generate the *main_visit* column.
      - This field has 3 possibilities: 
         - *Unknown:* We do not know if it is **main_visit** yet.
         - *Yes:* It was checked and visit is a **main_visit**.
         - *No:* It was checked and visit is NOT a **main_visit**.
       - We create this columns as a category. Initially all records are *Unknown*. 
   3. Generate the auxiliary columns:
      - `is_contained`, to mark completely contained records.
      - `is_partial`, to mark partialy contained records.
      - `is_not_contained`, to marked not contained records, which could be main visits.
      - `parent_visit_detail_id`, to record the associated **main_visit** when it is found.

The previous operations only need to be done once at the start.

Afterwards, we will need to do the following operations in a loop until no *Unknown* visits are left:

3. Identify the next batch of **main_visit** (`identify_next_main_visits()`):
   1. They are the most recent that verify:
      1. They are *"Unknown"*
      2. **visit_detail_id** == **visit_detail_id_original**
4. Check the if the other visits are:
   1. Completely contained (`identify_contained_rows()`):
      - **main_visit** -> *"No"*. 
      - Assign the value of **visit_detail_id** to the column **parent_visit_detail** (`update_contained_rows()`).
   2. Partially contained (`identify_partial_rows()`):
      - **main_visit** -> *"No"*. 
      - Assign the value of the latest **visit_detail_end_datetime** to the column **visit_end_datetime**. **(!)** Only for partially contained (`update_partial_rows()`).
   3. Not contained (`identify_not_contained_rows()`):
      - These records have to be reanalyzed. They are **main_visit** candidates.
      1. For not_contained records, we group by patient and extract columns **visit_detail_start_datetime**, **visit_detail_end_datetime** and **visit_detail_id** from the earliest *Unknown* record.
      2. Join with previous table, regenerating columns **visit_start_datetime**, **visit_end_datetime** and **visit_detail_id_original**.
5. Go back to step 3. Repeat until there are no more **main_visit** == *"Unknown"* columns or a safe iteration threshold is surpassed.
6. Assign a unique visit_occurrence_id to main visits.


#### Detailed step by step test

##### Test dataset creation

We are going to create a dataset with all the issues we can find.

- Posterior visits completely contained in a previous visit.
  * (2020-01-01, 2020-02-01) contains (2020-01-02, 2020-02-02) and (2020-01-04, 2020-02-04) 
    - We want to keep only the first.
- Posterior dates that are partially contained in a previous visit.
  * (2020-03-01, 2020-04-01) partially contains (2020-03-15, 2020-04-15) 
    - Here we want to combine both => (2020-03-01, 2020-04-15). 
    - "Multi-day visits must not overlap, i.e. share days other than start and end days". See [here](https://ohdsi.github.io/CommonDataModel/cdm54.html#visit_occurrence).
- Another issue is when we have several visits partially contained.
  - When we repeat the process, we need to make sure that previous main visits keep the end_dates
  * (2020-06-01, 2020-07-01) partially contains (2020-06-10, 2020-07-10) and (2020-06-20, 2020-07-20)
- Make sure we do not mix visits from different patient.

In [ ]:
import pandas as pd

column_names = [
    "person_id",
    "start_date",
    "end_date",
    "type_concept",
    "should_remain",
    "visit_concept_id",
    "provider_id",
]
filas = [
    # == Problema de fechas ==
    # We want two main visits, so we know we can distinguish between them
    # Each main visit will have a completely contained visit and a partially contained visit
    # Each main will have
    # -- Main visit 1 --
    (1, "2020-01-01", "2020-02-01", 1, True, 9202, 0),
    # Completely contained, same type_concept
    (1, "2020-01-02", "2020-01-02", 1, False, 9202, 0),
    # Completely contained, different type_concept
    (1, "2020-01-04", "2020-01-04", 2, False, 9202, 0),
    # Partially contained, same type_concept
    (1, "2020-01-06", "2020-02-06", 1, False, 9202, 0),
    # Partially contained, different type_concept
    (1, "2020-01-08", "2020-02-08", 2, False, 9202, 0),
    #
    # -- Main visit 2 --
    (1, "2020-03-01", "2020-04-01", 1, True, 9202, 0),
    # Completely contained, same type_concept
    (1, "2020-03-02", "2020-03-02", 1, False, 9202, 0),
    # Completely contained, different type_concept
    (1, "2020-03-04", "2020-03-04", 2, False, 9202, 0),
    # Partially contained, same type_concept
    (1, "2020-03-06", "2020-04-06", 1, False, 9202, 0),
    # Partially contained, different type_concept
    (1, "2020-03-08", "2020-04-08", 2, False, 9202, 0),
    #
    # == Problema de person_id ==
    # Tres personas distintas
    (2, "2021-01-01", "2021-01-01", 1, True, 9202, 0),
    (2, "2021-02-01", "2021-02-01", 1, True, 9202, 0),
    # Comparte fecha con 1
    (3, "2021-02-01", "2021-02-01", 2, True, 9202, 0),
    # No omparte fecha con 1
    (3, "2021-03-01", "2021-03-01", 2, True, 9202, 0),
    #
    # == Problema de type_concept ==
    (4, "2022-03-01", "2022-04-01", 1, True, 9202, 0),
    # Misma persona y fecha, sólo debe quedar type_concept == 1
    (4, "2022-03-01", "2022-04-01", 2, False, 9202, 0),
]
df_raw = pd.DataFrame.from_records(filas, columns=column_names)
df_raw["start_date"] = pd.to_datetime(df_raw["start_date"])
df_raw["end_date"] = pd.to_datetime(df_raw["end_date"])
(print(df_raw.info()))
df_raw

##### Code

In [ ]:
import polars as pl
import numpy as np

pl.Config(
    tbl_formatting="MARKDOWN",
    tbl_rows=20,
    set_tbl_width_chars=400,
    set_tbl_cols=-1,
)

df_raw_pl = pl.DataFrame(df_raw)
print(df_raw_pl)

1. Build the frame for the VISIT_DETAIL table (`build_visit_detail()`).
   1. Sort by `person_id`(asc), `start_date` (asc), `end_date` (desc) and `type_concept` (asc).
      * This way, we make sure that, for each `start_date`, the first column will have the latest `end_date`. This latest `end_date` is the visit that might contain other visits with the same `start_date`.
      * The `type_concept` column should be transformed first to a category datatype with a predefined order. This way the sorting will take that into account as well.
   2. Generate the field `visit_detail_id` with every visit.
   3. Rename columns to fit the VISIT_DETAIL structure.

In [ ]:
def build_visit_detail(df):
    return (
        # First we do the sorting
        df.sort(
            ["person_id", "start_date", "end_date", "type_concept"],
            descending=[False, False, True, False],
        )
        # Assign the visit_detail_id
        .with_columns(visit_detail_id=pl.int_range(pl.len()))
        # Rename columns
        .rename(
            {
                "start_date": "visit_detail_start_datetime",
                "end_date": "visit_detail_end_datetime",
                "type_concept": "visit_detail_type_concept_id",
            }
        )
    )


visit_detail = build_visit_detail(df_raw_pl)
print(visit_detail)

2. Extend the VISIT_DETAIL table to prepare for VISIT_OCCURRENCE generation (`build_visit_detail_extended()`):
   1. Generate placeholders for future VISIT_OCCURRENCE columns and for temporary columns.
      1. Group by each patient, generating:
        - **visit_start_datetime**: Initially the first **visit_detail_start_datetime**. It will be the **visit_start_datetime** VISIT_OCCCURRENCE column.
        - **visit_end_datetime**: Initially the first **visit_detail_end_datetime**. It will be the **visit_start_datetime** VISIT_OCCCURRENCE column. 
          - If visits needs to be joined together, this record will have the latest joined visit.
        - **visit_detail_id_original**: Initially the first **visit_detail_id**. It will be the **visit_detail_id** VISIT_OCCCURRENCE column.
        - **main_visit**: Initially will have a constant *"Unknown"* value for every record. It will be updated to identify main visits (Changed to "Yes") and the rest (Changed to "No").
      2. Join with VISIT_DETAIL to get the first row (earliest start_date, latest end_date) for each patient.
   2. Generate the *main_visit* column.
      - This field has 3 possibilities: 
         - *Unknown:* We do not know if it is **main_visit** yet.
         - *Yes:* It was checked and visit is a **main_visit**.
         - *No:* It was checked and visit is NOT a **main_visit**.
       - We create this columns as a category. Initially all records are *Unknown*. 
   3. Generate the auxiliary columns:
      - `is_contained`, to mark completely contained records.
      - `is_partial`, to mark partialy contained records.
      - `is_not_contained`, to marked not contained records, which could be main visits.
      - `parent_visit_detail_id`, to record the associated **main_visit** when it is found.

In [ ]:
def build_visit_detail_extended(visit_detail):
    # Get the first date of every person
    visit_occurrence_dates = visit_detail.group_by(
        "person_id", maintain_order=True
    ).agg(
        [
            pl.col("visit_detail_start_datetime").first().alias("visit_start_datetime"),
            pl.col("visit_detail_end_datetime").first().alias("visit_end_datetime"),
            pl.col("visit_detail_id").first().alias("visit_detail_id_original"),
        ]
    )

    # Join and return
    return visit_detail.join(
        visit_occurrence_dates, on="person_id", how="left"
    ).with_columns(
        main_visit=pl.lit("Unknown").cast(pl.Enum(["Yes", "No", "Unknown"])),
        is_contained=pl.lit(False),
        is_partial=pl.lit(False),
        not_contained=pl.lit(False),
        parent_visit_detail_id=pl.lit(None),
    )


df = build_visit_detail_extended(visit_detail)
print(df)

3. Identify the next batch of **main_visit** (`identify_next_main_visits()`):
   1. They are the most recent that verify:
      1. They are *"Unknown"*
      2. **visit_detail_id** == **visit_detail_id_original**

In [ ]:
# Identify main_visits
def identify_next_main_visits(df):
    return df.with_columns(
        main_visit=(
            pl.when(
                (pl.col("visit_detail_id_original") == pl.col("visit_detail_id"))
                & (pl.col("main_visit") == "Unknown")
            )
            .then(pl.lit("Yes"))
            .otherwise(pl.col("main_visit"))
        )
    )


df = identify_next_main_visits(df)
df

4. Check the if the other visits are:

   1. Completely contained (`identify_contained_rows()`):
      - **main_visit** -> *"No"*. 
      - Assign the value of **visit_detail_id** to the column **parent_visit_detail** (`update_contained_rows()`).

In [ ]:
def identify_contained_rows(df):
    return df.with_columns(
        is_contained=pl.when(
            (pl.col("main_visit") == "Unknown")
            & (pl.col("visit_start_datetime") <= pl.col("visit_detail_start_datetime"))
            & (pl.col("visit_end_datetime") >= pl.col("visit_detail_end_datetime"))
        )
        .then(True)
        .otherwise(pl.col("is_contained"))
    )


def update_contained_rows(df):

    return df.with_columns(
        # Update main_visit to mark contained visits
        main_visit=(
            pl.when((pl.col("is_contained") == True))
            .then(pl.lit("No"))
            .otherwise(pl.col("main_visit"))
        ),
        # Build the parent_visit_detail_id, since we are here
        parent_visit_detail_id=(
            pl.when(
                (pl.col("is_contained") == True)
                & (pl.col("visit_detail_id") != pl.col("visit_detail_id_original"))
            )
            .then(pl.col("visit_detail_id_original"))
            .otherwise(pl.col("parent_visit_detail_id"))
        ),
    )


df = identify_contained_rows(df)
df = update_contained_rows(df)
df

4. Check the if the other visits are:
   
   2. Partially contained (`identify_partial_rows()`):
      - **main_visit** -> *"No"*. 
      - Assign the value of the latest **visit_detail_end_datetime** to the column **visit_end_datetime**. **(!)** Only for partially contained (`update_partial_rows()`).


In [ ]:
def identify_partial_rows(df):
    return df.with_columns(
        is_partial=pl.when(
            (pl.col("main_visit") == "Unknown")
            & (pl.col("visit_detail_start_datetime") <= pl.col("visit_end_datetime"))
            & (pl.col("visit_detail_end_datetime") > pl.col("visit_end_datetime"))
        )
        .then(True)
        .otherwise(pl.col("is_partial"))
    )


def update_partial_rows(df):

    latest_date = (
        df.filter(pl.col("is_partial") == True)
        .group_by("person_id", maintain_order=True)
        .agg(pl.col("visit_detail_end_datetime").max().alias("latest_end_datetime"))
    )

    return (
        # Join back to the main dataframe and update visit_end_datetime
        df.join(
            latest_date,
            on="person_id",
            how="left",
        )
        .with_columns(
            main_visit=(
                pl.when((pl.col("is_partial") == True))
                .then(pl.lit("No"))
                .otherwise(pl.col("main_visit"))
            ),
            visit_end_datetime=pl.when(pl.col("main_visit") == "Yes")
            .then(
                pl.coalesce(
                    [pl.col("latest_end_datetime"), pl.col("visit_detail_end_datetime")]
                )
            )
            .otherwise(pl.col("visit_end_datetime")),
        )
        .drop("latest_end_datetime")
    )


df = identify_partial_rows(df)
df = update_partial_rows(df)
df

4. Check the if the other visits are:

   3. Not contained (`identify_not_contained_rows()`):
      - These records have to be reanalyzed. They are **main_visit** candidates.

      1. For not_contained records, we group by patient and extract columns **visit_detail_start_datetime**, **visit_detail_end_datetime** and **visit_detail_id** from the earliest (first) *Unknown* record.
      2. Join with previous table, regenerating columns **visit_start_datetime**, **visit_end_datetime** and **visit_detail_id_original**.

In [ ]:
def identify_not_contained_rows(df):
    return df.with_columns(
        not_contained=pl.when(
            (pl.col("main_visit") == "Unknown")
            & (pl.col("visit_detail_start_datetime") >= pl.col("visit_end_datetime"))
        )
        .then(True)
        .otherwise(pl.col("not_contained"))
    )


def update_not_contained_rows(df):

    newest_not_contained = (
        df.filter(pl.col("not_contained") == True)
        .group_by("person_id", maintain_order=True)
        .agg(
            pl.col("visit_detail_id").first().alias("visit_detail_id_newest"),
            pl.col("visit_detail_start_datetime")
            .first()
            .alias("visit_detail_start_datetime_newest"),
            pl.col("visit_detail_end_datetime")
            .first()
            .alias("visit_detail_end_datetime_newest"),
        )
    )

    return (
        # Join back to the main dataframe and update visit_end_datetime
        df.join(
            newest_not_contained,
            on="person_id",
            how="left",
        )
        # Update the values on visit_detail_id_original, visit_start_datetime and visit_end_datetime
        .with_columns(
            visit_detail_id_original=pl.when((pl.col("main_visit") == "Unknown"))
            .then(pl.col("visit_detail_id_newest"))
            .otherwise(pl.col("visit_detail_id_original")),
            visit_start_datetime=pl.when((pl.col("main_visit") == "Unknown"))
            .then(pl.col("visit_detail_start_datetime_newest"))
            .otherwise(pl.col("visit_start_datetime")),
            visit_end_datetime=pl.when((pl.col("main_visit") == "Unknown"))
            .then(pl.col("visit_detail_end_datetime_newest"))
            .otherwise(pl.col("visit_end_datetime")),
        ).drop(
            pl.col(
                "visit_detail_id_newest",
                "visit_detail_start_datetime_newest",
                "visit_detail_end_datetime_newest",
            )
        )
    )


df = identify_not_contained_rows(df)
df = update_not_contained_rows(df)
df

5. Go back to step 3. Repeat until there are no more **main_visit** == *"Unknown"* columns or a safe iteration threshold is surpassed.


In [ ]:
# Identify main_visits
df = identify_next_main_visits(df)
# Update completely contained rows
df = identify_contained_rows(df)
df = update_contained_rows(df)
# Update partially contained rows
df = update_partial_rows(df)
df = identify_partial_rows(df)
# Update not contained rows
df = update_not_contained_rows(df)
df = identify_not_contained_rows(df)
print(df)

6. Assign a unique visit_occurrence_id to main visits.

In [ ]:
def assign_visit_occurrence_id(visit_occurrence):

    return (
        # Create a helper column to track main visit sequence
        visit_occurrence.with_columns(is_main_visit=(pl.col("main_visit") == "Yes"))
        # Assign a unique identifier only to main visits using row_number and clean the helper
        .with_columns(
            visit_occurrence_id=pl.when(pl.col("is_main_visit"))
            .then(pl.col("is_main_visit").cast(pl.Int32).cum_sum() - 1)
            .otherwise(None)
        ).drop("is_main_visit")
        # Fill the rest using forward fill (ffill)
        .with_columns(visit_occurrence_id=pl.col("visit_occurrence_id").forward_fill())
    )


visit_occurrence = assign_visit_occurrence_id(df)
visit_occurrence

We need to build the VISIT_DETAIL and VISIT_OCCURRENCE table by removing all the helper columns we have created

In [ ]:
def build_visit_occurrence(df, verbose=0, n_iter_max=1000):
    # -- Initialization --
    df = df.lazy()
    # Get the core of the visit_detail table
    df = build_visit_detail(df)
    # Extend the table for processing
    df = build_visit_detail_extended(df)
    # Initialize counters for the while loop
    n_unknown = (
        df.filter(pl.col("main_visit") == "Unknown").select(pl.len()).collect().item()
    )
    n_iter = 0
    df = df.collect()

    # -- Loop --
    while n_unknown > 0 and n_iter < n_iter_max:
        if verbose > 0:
            print(f"Iter {n_iter:>2}: {n_unknown} unknown rows left.")
        if verbose > 1:
            if isinstance(df, pl.LazyFrame):
                print(df.filter(pl.col("main_visit") == "Unknown").head(5).collect())
            else:
                print(df.filter(pl.col("main_visit") == "Unknown").head(5))

        # Look for next batch of main_visits
        df = identify_next_main_visits(df)

        # Identify and update completely contained visits
        df = identify_contained_rows(df)
        df = update_contained_rows(df)

        # Identify and update partially contained visits
        df = identify_partial_rows(df)
        df = update_partial_rows(df)

        # Identify and update not contained visits
        df = identify_not_contained_rows(df)
        df = update_not_contained_rows(df)

        # Update conditions
        if isinstance(df, pl.LazyFrame):
            n_unknown = (
                df.filter(pl.col("main_visit") == "Unknown")
                .select(pl.len())
                .collect()
                .item()
            )
        else:
            n_unknown = (
                df.filter(pl.col("main_visit") == "Unknown").select(pl.len()).item()
            )
        n_iter += 1

    # Assign an unique visit_occurrence_id only to main_visits
    df = assign_visit_occurrence_id(df)

    # Drop the extra helper columns
    df = df.drop(
        # Drop helpers
        pl.col("visit_detail_id_original"),
        pl.col("is_contained"),
        pl.col("is_partial"),
        pl.col("not_contained"),
    )

    # -- Build the core of the visit_detal table --
    visit_detail = df

    # Drop visit_occurrence columns
    visit_detail = visit_detail.drop(
        pl.col("visit_start_datetime"),
        pl.col("visit_end_datetime"),
        pl.col(
            "main_visit"
        ),  # This one is dropped here so it can be used for visit_occurrence
    )

    # -- Build the core of the visit_occurrence table --
    visit_occurrence = df

    # Get only main visits
    visit_occurrence = df.filter(pl.col("main_visit") == "Yes").drop(
        pl.col("main_visit")
    )

    # Rename columns
    visit_occurrence = visit_occurrence.rename(
        {
            "visit_detail_type_concept_id": "visit_type_concept_id",
        }
    )

    # Drop columns from visit_detail
    visit_occurrence = visit_occurrence.drop(
        pl.col("visit_detail_start_datetime"),
        pl.col("visit_detail_end_datetime"),
        pl.col("visit_detail_id"),
        pl.col("parent_visit_detail_id"),
    )

    # n_unknown = df.filter(pl.col("main_visit") == "Unknown").select(pl.len()).collect().item() # Lazy
    n_unknown = (
        df.filter(pl.col("main_visit") == "Unknown").select(pl.len()).item()
    )  # Not Lazy

    # return visit_detail.collect(), visit_occurrence.collect()
    return visit_detail, visit_occurrence


visit_detail, visit_occurrence = build_visit_occurrence(df_raw_pl, verbose=1)

In [ ]:
visit_detail

In [ ]:
visit_occurrence

In [ ]:
# Check that visit_detail has the correct visit_occurrence_id asssignation
assert (
    np.all(
        visit_detail.select("visit_occurrence_id")
        == pl.Series([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 3, 4, 5, 6, 6])
    )
    == True
)
# Check that visit_occurrence has the correct shape
assert visit_occurrence.shape[0] == 7
# Check that visit_occurrence has the correct people
assert (
    np.all(visit_occurrence.select("person_id") == pl.Series([1, 1, 2, 2, 3, 3, 4]))
    == True
)
# Check that visit_occurrence has the same start dates
assert (pl.Series(
    [
        "2020-01-01 00:00:00",
        "2020-03-01 00:00:00",
        "2021-01-01 00:00:00",
        "2021-02-01 00:00:00",
        "2021-02-01 00:00:00",
        "2021-03-01 00:00:00",
        "2022-03-01 00:00:00",
    ]
).str.to_datetime() == visit_occurrence["visit_start_datetime"]).all()
# Check that visit_occurrence has the same start dates
assert (pl.Series(
    [
        "2020-02-08 00:00:00",
        "2020-04-08 00:00:00",
        "2021-01-01 00:00:00",
        "2021-02-01 00:00:00",
        "2021-02-01 00:00:00",
        "2021-03-01 00:00:00",
        "2022-04-01 00:00:00",
    ]
).str.to_datetime() == visit_occurrence["visit_end_datetime"]).all()

In [ ]:
%timeit -n 10 -r 5 build_visit_occurrence(df_raw_pl)

- 20.9 ms ± 2.45 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
- 65.2 ms ± 6.47 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
- 46.7 ms ± 3.65 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
  - Creo que está afectando el que el clúster esté saturadillo. 
- 21.3 ms ± 2.27 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
- 20.8 ms ± 1.22 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)



TODO:

- Verificar tests.
- Hacer benchmarks
- Limpiar este notebook para que sólo tenga la explicación del código actual
  - Traducir a inglés.

#### Test piping

In [ ]:
df = df_raw_pl.lazy()
visit_detail, visit_occurrence = build_visit_occurrence(df)

## 1.2 Old Recursive approach

In [ ]:
import warnings
import pandas as pd


def find_overlap_index(df: pd.DataFrame) -> pd.Series:
    """Finds all rows that:
       - belong to the same person_id
       - are contained with the previous row.
       - are not single day visits
    and removes them.

    Parameters
    ----------
    df : pd.DataFrame
        pandas Dataframe with at least four columns.
        Assumes first column is person_id, second column is
        start_date and third column is end_date.

    Returns
    -------
    pd.Series
        pandas Series with bools. True if row is contained
        with the previous row, False otherwise.
    """
    # 1. Check that current and previous patient are the same
    idx_person = df.iloc[:, 0] == df.iloc[:, 0].shift(1)
    # 2. Check that current start_date is later that previous start_date
    idx_start = df.iloc[:, 1] >= df.iloc[:, 1].shift(1)
    # 3. Check that current end_date is sooner that previous end_date
    idx_end = df.iloc[:, 2] <= df.iloc[:, 2].shift(1)
    # 4. Check that current interval and previos interval are not both single_day
    interval = df.iloc[:, 2] - df.iloc[:, 1]
    idx_int_curr = interval <= pd.Timedelta(1, unit="D")
    idx_int_prev = interval.shift(1) <= pd.Timedelta(1, unit="D")
    idx_interval = ~(idx_int_curr & idx_int_prev)
    # 5. If everything past is true, I can drop the row
    return idx_start & idx_end & idx_person & idx_interval


def remove_overlap(
    df: pd.DataFrame,
    sorting_columns: tuple,
    ascending_order: tuple,
    verbose: int = 0,
    _counter: int = 0,
    _counter_lim: int = 1000,
) -> pd.DataFrame:
    """Removes all rows that are completely contained within
    another row. It will not remove rows that are only partially
    contained within the previous one.

    The function works by sorting the rows by columns. If two or
    more rows are overlapping, only the top one will be kept.


    Parameters
    ----------
    df : pd.DataFrame
        pandas dataframe with overlapping rows to be removed.
        Selection of columns is done by selecting ncols in order.
        This allows its use for different tables with columns
        that have the same purpose but different names.
    sorting_columns : tuple
        Columns to use for sorting.
        Usually, expects 4 columns: 'person_id', 'start_date', 'end_date'
        and some '*_concept_id', like 'visit_concept_id'.
    ascending_order : tuple
        List of bools indicating if each row should have ascending or descending
        order.
        Important! Usually all are true except end_date column. See Notes.
    verbose : int, optional, default 0
        Information output
        - 0 No info
        - 2 Show number of iterations
        - 3 Show an example of the first row being removed and
            the row that contains it.
    _counter : int
        Iteration control param. Number of iterations.
        0 will be used to begin and function will take over.
    _counter_lim : int, optional, default 1000
        Iteration control param. Limit of iterations

    Returns
    -------
    pd.DataFrame
        Copy of input dataframe with contained rows removed.

    Notes
    -------
    The usual behavior is to have 'person_id', 'start_date' and 'end_date'
    as first columns, in ascending, ascending and descending order, respectively.
    This ensures that:
    - All records for the same person are together (sorting by person_id first)
    - Earlier records are placed at the top (sorting by ascending start_date)
    - Longer duration visits are placed at the top (sorting by descending end_date)

    Bear in mind that missing values will be placed at the bottom by default. Any extra
    columns provided will leave any missing values out in case of overlapping records.
    """
    # == Preparation =================================================
    # Sanity checks
    if len(sorting_columns) != len(ascending_order):
        raise ValueError(
            "'sorting_columns' and 'ascending_order' lengths must be equal."
        )

    cond_sort = sorting_columns[:3] != ["person_id", "start_date", "end_date"]
    cond_asce = ascending_order[:3] != [True, True, False]
    if cond_sort or cond_asce:
        warnings.warn(
            "Sorting and ascending initial columns are not the expected order. \
                 Make sure data output is correct."
        )

    # Sort the dataframe if first iteration
    if _counter == 0:
        if verbose > 0:
            print("Removing overlapping rows...")
        if verbose > 1:
            print(f" Iter 0 => {df.shape[0]} initial rows.")
        df = df.sort_values(sorting_columns, ascending=ascending_order)

    # == Find indexes ================================================
    # Get the rows
    idx_to_remove = find_overlap_index(df)

    # == Main "loop" =================================================
    # Prepare next loop
    idx_to_remove_sum = idx_to_remove.sum()
    _counter += 1
    # If there's still room to go, go
    if (idx_to_remove_sum != 0) and (_counter < _counter_lim):
        if verbose > 1:
            # Show iteration and number of rows removed
            print(f" Iter {_counter} => {idx_to_remove_sum} rows removed.")
        if verbose > 2:
            # Get first removed row and show container and contained row
            idx_max = df.index.get_loc(idx_to_remove.idxmax())
            print(f"{df.iloc[(idx_max-1):idx_max+1, :4]}")
        return remove_overlap(
            df.loc[~idx_to_remove], sorting_columns, ascending_order, verbose, _counter
        )
    else:
        return df

In [ ]:
import sys
import pyarrow as pa

sys.path.append("../../")
import bps_to_omop.utils.process_dates as pro_dat

table_raw = pa.Table.from_pandas(df_raw)
table_raw = table_raw.cast(
    pa.schema(
        [
            ("person_id", pa.int64()),
            ("start_date", pa.date64()),
            ("end_date", pa.date64()),
            ("type_concept", pa.int64()),
            ("should_remain", pa.int64()),
            ("visit_concept_id", pa.int64()),
            ("provider_id", pa.int64()),
        ]
    )
)
# Define sorting order
sorting_columns = ["person_id", "start_date", "end_date", "visit_concept_id"]
ascending_order = [True, True, False, True]

df_rare = table_raw.to_pandas()
df_done = remove_overlap(df_rare, sorting_columns, ascending_order, verbose=1)
df_done.sort_index()

In [ ]:
%timeit -n 10 -r 5 remove_overlap(df_rare, sorting_columns, ascending_order, verbose=0)

### Adapt polars to remove_overlap

The old method did something wrong, it did not joined together dates that were partially contained. Some rows that should be removed are not.

To measure speed with big datasets, we need to verify that both functions return the same results. To do this, we will adapt the functions created for the polars use to match the expected result of the old implementation. 

In [ ]:
# Retrieve the latest visit_detail_end_datetime from the partially contained visits
def retrieve_newest_not_contained_adapted(df):
    newest_not_contained = (
        df.filter(pl.col("main_visit") == "Unknown")
        .group_by("person_id")
        .agg(
            pl.col("visit_detail_id").first().alias("visit_detail_id_newest"),
            pl.col("visit_detail_start_datetime")
            .first()
            .alias("visit_detail_start_datetime_newest"),
            pl.col("visit_detail_end_datetime")
            .first()
            .alias("visit_detail_end_datetime_newest"),
        )
    )
    return newest_not_contained


def update_not_contained_rows_adapted(df):

    newest_not_contained = retrieve_newest_not_contained_adapted(df)

    # Join back to the main dataframe and update visit_end_datetime
    df = df.join(
        newest_not_contained,
        on="person_id",
        how="left",
    )
    # Update the values on visit_detail_id_original, visit_start_datetime and visit_end_datetime
    df = df.with_columns(
        visit_detail_id_original=pl.when((pl.col("main_visit") == "Unknown"))
        .then(pl.col("visit_detail_id_newest"))
        .otherwise(pl.col("visit_detail_id_original")),
        visit_start_datetime=pl.when((pl.col("main_visit") == "Unknown"))
        .then(pl.col("visit_detail_start_datetime_newest"))
        .otherwise(pl.col("visit_start_datetime")),
        visit_end_datetime=pl.when((pl.col("main_visit") == "Unknown"))
        .then(pl.col("visit_detail_end_datetime_newest"))
        .otherwise(pl.col("visit_end_datetime")),
    ).drop(
        pl.col(
            "visit_detail_id_newest",
            "visit_detail_start_datetime_newest",
            "visit_detail_end_datetime_newest",
        )
    )
    return df


def remove_overlap_polars(df, verbose=0):
    # -- Initialization --
    # Get the core of the visit_detail table
    df = build_visit_detail(df)
    # Extend the table for processing
    df = build_visit_detail_extended(df)
    # Initialize counters for the while loop
    n_unknown = df.filter(pl.col("main_visit") == "Unknown").height
    n_iter = 0

    # -- Loop --
    while n_unknown > 0 and n_iter < 1000:
        if verbose > 0:
            print(f"Iter {n_iter:>2}: {n_unknown} unknown rows left.")
        if verbose > 1:
            print(df.head(20))

        # Look for next batch of main_visits
        df = identify_next_main_visits(df)

        # Identify and mark completely contained visits
        df = identify_contained_rows(df)
        df = update_contained_rows(df)

        # Identify and mark partially contained visits
        df = identify_partial_rows(df)
        df = update_partial_rows(df)

        # Identify and mark not contained visits
        df = identify_not_contained_rows(df)
        df = update_not_contained_rows(df)

        # Update conditions
        n_unknown = df.filter(pl.col("main_visit") == "Unknown").height
        n_iter += 1

    # Assign an unique visit_occurrence_id only to main_visits
    df = assign_visit_occurrence_id(df)

    # Drop the extra helper columns
    df = df.drop(
        # Drop helpers
        pl.col("visit_detail_id_original"),
        pl.col("is_contained"),
        pl.col("is_partial"),
        pl.col("not_contained"),
    )

    # -- Build the core of the visit_detal table --
    visit_detail = df

    # Drop visit_occurrence columns
    visit_detail = visit_detail.drop(
        pl.col("visit_start_datetime"),
        pl.col("visit_end_datetime"),
        pl.col(
            "main_visit"
        ),  # This one is dropped here so it can be used for visit_occurrence
    )

    # -- Build the core of the visit_occurrence table --
    visit_occurrence = df

    # Get only main visits
    visit_occurrence = df.filter(pl.col("main_visit") == "Yes").drop(
        pl.col("main_visit")
    )

    # Rename columns
    visit_occurrence = visit_occurrence.rename(
        {
            "visit_detail_type_concept_id": "visit_type_concept_id",
        }
    )

    # Drop columns from visit_detail
    visit_occurrence = visit_occurrence.drop(
        pl.col("visit_detail_start_datetime"),
        pl.col("visit_detail_end_datetime"),
        pl.col("visit_detail_id"),
        pl.col("parent_visit_detail_id"),
    )

    return visit_detail, visit_occurrence


_visit_detail, visit_occurrence_polars = remove_overlap_polars(df_raw_pl, verbose=1)
visit_occurrence_polars = visit_occurrence_polars.select(
    "person_id",
    "visit_start_datetime",
    "visit_end_datetime",
    "visit_type_concept_id",
    "should_remain",
    "visit_concept_id",
    "provider_id",
)

visit_occurrence_polars = visit_occurrence_polars.with_columns(
    pl.col("visit_start_datetime").dt.to_string().str.to_datetime(),
    pl.col("visit_end_datetime").dt.to_string().str.to_datetime(),
)

print(visit_occurrence_polars)

In [ ]:
# Get the recursive approach here
visit_occurrence_recursive = remove_overlap(
    df_rare, sorting_columns, ascending_order, verbose=2
)
visit_occurrence_recursive = (
    visit_occurrence_recursive.reset_index(drop=True)
    .sort_values(["person_id", "start_date", "end_date", "visit_concept_id"])
    .rename(
        {
            "start_date": "visit_start_datetime",
            "end_date": "visit_end_datetime",
            "type_concept": "visit_type_concept_id",
        },
        axis=1,
    )
)

visit_occurrence_recursive = pl.DataFrame(visit_occurrence_recursive)
visit_occurrence_recursive = visit_occurrence_recursive.with_columns(
    pl.col("visit_start_datetime").dt.to_string().str.to_datetime(),
    pl.col("visit_end_datetime").dt.to_string().str.to_datetime(),
    pl.col("should_remain").cast(pl.Boolean),
)
print(visit_occurrence_recursive)

In [ ]:
from polars.testing import assert_frame_equal

# assert_frame_equal(visit_occurrence_recursive, visit_occurrence_polars)

## 1.3 Test with big datasets

Vamos a comparar la velocidad de ambos métodos con datasets grandes.

Nos traemos la función para generar datasets

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
from pyarrow import parquet


def create_sample_df(
    n: int = 1000,
    n_dates: int = 50,
    first_date: str = "2020-01-01",
    last_date: str = "2023-01-01",
    mean_duration_days: int = 60,
    std_duration_days: int = 180,
) -> pd.DataFrame:

    # == Parameters ==
    np.random.seed(42)
    pd.options.mode.string_storage = "pyarrow"
    # Start date from which to start the dates
    first_date = pd.to_datetime(first_date)
    last_date = pd.to_datetime(last_date)
    max_days = (last_date - first_date).days

    # == Generate IDs randomly ==
    # -- Generate the Ids
    people = np.random.randint(10000000, 99999999 + 1, size=n)
    person_id = np.random.choice(people, n * n_dates)

    # == Generate random dates ==
    # Generate random integers for days and convert to timedelta
    random_days = np.random.randint(0, max_days, size=n * n_dates)
    # Create the columns
    observation_start_date = first_date + pd.to_timedelta(random_days, unit="D")
    # Generate a gaussian sample of dates
    random_days = np.random.normal(
        mean_duration_days, std_duration_days, size=n * n_dates
    )
    random_days = np.int32(random_days)
    observation_end_date = observation_start_date + pd.to_timedelta(
        random_days, unit="D"
    )
    # Correct end_dates
    # => If they are smaller than start_date, take start_date
    observation_end_date = np.where(
        observation_end_date < observation_start_date,
        observation_start_date,
        observation_end_date,
    )

    # == Generate the code ==
    period_type_concept_id = np.random.randint(1, 11, size=n * n_dates)

    # == Generate the dataframe ==
    df_raw = {
        "person_id": person_id,
        "observation_period_start_date": observation_start_date,
        "observation_period_end_date": observation_end_date,
        "period_type_concept_id": period_type_concept_id,
    }
    return pd.DataFrame(df_raw)

### Time test

In [ ]:
# Cargamos los datos
df_raw = create_sample_df(n=1000)
df_raw.columns = ["person_id", "start_date", "end_date", "type_concept"]
df_raw.loc[:, "visit_concept_id"] = 9202

df_raw_pl = pl.DataFrame(df_raw)

In [ ]:
# print('\nrecursive:')
# %timeit -n 10 -r 5 remove_overlap(df_raw, sorting_columns, ascending_order, verbose=0)

# print('\npolars test:')
# %timeit -n 10 -r 5 build_visit_occurrence(df_raw_pl, verbose=0)

For n = 100 000

    recursive:
    3.66 s ± 68.8 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)

    polars optimized :
    1.08 s ± 50.2 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)

For n = 1000

    recursive:
    34.2 ms ± 482 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)

    polars optimized :
    25.9 ms ± 558 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)

    polars test:
    7.72 s ± 111 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)

    polars island:
    4.06 s ± 377 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)

In [ ]:
# Get the core of the visit_detail table
df = build_visit_detail(df_raw_pl)
# Extend the table for processing
df = build_visit_detail_extended(df)
df

In [ ]:
# Identify main_visits
df = identify_next_main_visits(df)
# Update completely contained rows
df = identify_contained_rows(df)
df = update_contained_rows(df)
# Update partially contained rows
df = identify_partial_rows(df)
df = update_partial_rows(df)
# Update not contained rows
df = identify_not_contained_rows(df)
df = update_not_contained_rows(df)
df

In [ ]:
# Identify main_visits
df = identify_next_main_visits(df)
df

In [ ]:
# Update completely contained rows
df = identify_contained_rows(df)
df = update_contained_rows(df)
df

In [ ]:
# Update partially contained rows
df = identify_partial_rows(df)
df = update_partial_rows(df)
df

In [ ]:
# Update not contained rows
df = identify_not_contained_rows(df)
df = update_not_contained_rows(df)
df

In [ ]:
visit_detail, visit_occurrence = build_visit_occurrence(
    df_raw_pl, verbose=2, n_iter_max=4
)

# 2. Eliminar filas cercanas
Una vez que las filas contenidas en otras se han eliminado, el objetivo ahora es eliminar aquellas que están separadas por un número de días menor al que estipulemos.

## 2.1 Creación dataset de prueba
Vamos a suponer que vamos a agrupar aquellas fechas a menos de 1 año (365 días exactamente) una de otra. 

In [ ]:
import numpy as np
import pandas as pd

nombre_columnas = ["person_id", "start_date", "end_date", "type_concept"]
filas = [
    # Estas fechas deberían juntarse porque están a menos de 365 dias
    # type_concept debería ser 2
    (1, "2020-01-01", "2020-02-01", 1),
    (1, "2020-03-01", "2020-04-01", 2),
    (1, "2020-05-01", "2020-12-01", 2),
    # Esta última de la misma persona no
    (1, "2022-01-01", "2022-01-01", 2),
    # Estas fechas deberían juntarse porque se pisan
    # type_concept debería ser 1
    (2, "2020-01-01", "2020-06-01", 1),
    (2, "2020-03-01", "2020-09-01", 1),
    (2, "2020-06-01", "2020-12-01", 2),
    # Estas dos fechas NO deberían juntarse,
    # cada uno es su propio periodo
    (3, "2021-01-01", "2021-01-01", 1),
    (3, "2023-02-01", "2023-02-01", 2),
    (3, "2024-03-01", "2024-04-01", 3),
    # Se juntarían pero no porque son personas distintas
    (4, "2024-01-01", "2024-02-01", 1),
    (5, "2025-01-01", "2025-02-01", 2),
    # Deberían juntarse porque entras ellas hay poca distancia,
    # pero si eliminas una de golpe las otras están muy
    # separadas y no se juntan.
    # type_concept debería ser 2
    (6, "2020-01-01", "2020-12-01", 1),
    (6, "2021-01-01", "2021-12-01", 2),
    (6, "2022-01-01", "2022-12-01", 2),
    (6, "2023-01-01", "2023-12-01", 2),
]
df_raw = pd.DataFrame.from_records(filas, columns=nombre_columnas)
df_raw["start_date"] = pd.to_datetime(df_raw["start_date"])
df_raw["end_date"] = pd.to_datetime(df_raw["end_date"])
df_raw

Suponiendo que **agrupamos periodos separados por menos de 365 días** y que **usamos la moda para calcular el `type_concept` final**. El resultado del agrupamiento de los datos creados debería ser el siguiente:

In [ ]:
nombre_columnas = ["person_id", "start_date", "end_date", "type_concept"]
filas = [
    # Estas fechas deberían juntarse porque están a menos de 365 dias
    # type_concept debería ser 2
    (1, "2020-01-01", "2020-12-01", 2),
    # Esta última de la misma persona no
    (1, "2022-01-01", "2022-01-01", 2),
    # Estas fechas deberían juntarse porque se pisan
    # type_concept debería ser 1
    (2, "2020-01-01", "2020-12-01", 1),
    # Estas dos fechas NO deberían juntarse,
    # cada uno es su propio periodo
    (3, "2021-01-01", "2021-01-01", 1),
    (3, "2023-02-01", "2023-02-01", 2),
    (3, "2024-03-01", "2024-04-01", 3),
    # Se juntarían pero no porque son personas distintas
    (4, "2024-01-01", "2024-02-01", 1),
    (5, "2025-01-01", "2025-02-01", 2),
    # Deberían juntarse porque entras ellas hay poca distancia,
    # pero si eliminas una de golpe las otras están muy
    # separadas y no se juntan.
    # type_concept debería ser 2
    (6, "2020-01-01", "2023-12-01", 2),
]
df_result = pd.DataFrame.from_records(filas, columns=nombre_columnas)
df_result["start_date"] = pd.to_datetime(df_result["start_date"])
df_result["end_date"] = pd.to_datetime(df_result["end_date"])
df_result

## 2.2 Eliminación de filas cercanas

Hay varios problemas
- Si te encuentras varias filas que cumplen la condición seguidas, puedes perder información si la primera y la última filas están muy separadas.
- No se ha encontrado una manera efectiva de hacer esto sin iterar como antes.
    - O bien iteras por personas, y no tienes que vigilar que mezclas personas
    - O bien lo haces de golpe, pero es muy complejo llevar la cuenta de los `type_concept` y `person_id` que has eliminado


### 2.2.1 Usando sólo índices

Hay que encontrar la primera y última fila de cada persona y también aquellos casos en los que sólo haya una única fila.

Luego hay que buscar también aquellos casos en los que la siguiente fila esté muy alejada, lo que implicaría que hemos encontrado una brecha en el periodo de observación.

In [ ]:
# Get the raw files
df_rare = df_raw.sort_values(
    ["person_id", "start_date", "end_date"], ascending=[True, True, False]
)
# It is VERY important to reset the index to make sure we can
# retrieve them realiably after sorting them.
df_rare = df_rare.reset_index(drop=True)

# Create index for first, last or only person in dataset
df_rare["idx_person_first"] = (
    df_rare["person_id"] == df_rare["person_id"].shift(-1)
) & (df_rare["person_id"] != df_rare["person_id"].shift(1))
df_rare["idx_person_last"] = (
    df_rare["person_id"] != df_rare["person_id"].shift(-1)
) & (df_rare["person_id"] == df_rare["person_id"].shift(1))
df_rare["idx_person_only"] = (
    df_rare["person_id"] != df_rare["person_id"].shift(-1)
) & (df_rare["person_id"] != df_rare["person_id"].shift(1))
# Create index if the break is too big and needs to be kept
n_days = 365
df_rare["next_interval"] = df_rare["start_date"].shift(-1) - df_rare["end_date"]
df_rare["idx_interval"] = df_rare["next_interval"] >= pd.Timedelta(n_days, unit="D")
# Combine all to see which rows remain
df_rare["to_remain"] = (
    df_rare["idx_person_first"]
    | df_rare["idx_person_last"]
    | df_rare["idx_person_only"]
    | df_rare["idx_interval"]
)

df_rare

De esta gente me tengo que quedar seguro :
- `idx_person_first == True`
- `idx_person_last == True`
- `idx_person_only == True`

1. Si para una persona sólo hay 1 `idx_person_only == True`, me quedo ese y a correr. En este caso no hay que hacer nada, esa fila tiene la primera y la última fecha del paciente.

2. Si para una persona sólo hay 1 `idx_person_first == True` y 1 `idx_person_last == True`, entonces tengo el principio y el final. 
    1. Si no hay ningún `idx_interval == True`, junto la `start_date` del `idx_person_first == True` y la `end_date` del `idx_person_last == True`.
    2. Si hay algún `idx_interval == True`, tengo que tener en cuenta que esas filas indican brechas en el periodo de observación. La fila donde `idx_interval == True` indica que es la última del periodo y que la siguiente es el comienzo de otro.

Básicamente hay que registrar por un lado las `start_date`, con sus respectivos `person_id`, y por otro lado las nuevas `end_date`. Vamos a escribir el código que registra esto.

In [ ]:
# We will create an initial dataframe with only person_id and
# start_date. The end_date rows and type_concept will be added
# later as new columns.

# == start_date and person_id ==========================================
# To retrieve the start_date we need the indexes of:
# - single day periods (idx_person_only == True)
# - first dates (idx_person_first == True)
# - Rows just after period breaks, (idx_interval.index + 1)

# Get the person condition indexes
idx_start = df_rare.index[
    df_rare["idx_person_only"]
    | df_rare["idx_person_first"]
    | df_rare["idx_interval"].shift(1)
]
# Get the interval indexes
df_done = df_rare.loc[idx_start, ["person_id", "start_date"]]
df_done

# == end_date ==========================================================
# Get the indexes
idx_end = df_rare.index[
    df_rare["idx_person_only"] | df_rare["idx_person_last"] | df_rare["idx_interval"]
]
# Append values found to final dataframe
df_done["end_date"] = df_rare.loc[idx_end, ["end_date"]].values

df_done

Para encontrar los `type_concept`, podemos usar los índices del principio y el final, hacer un zip y, como deberían estar en orden. Tendré una lista con las parejas inicio final de cada periodo.

Si busco todos los `type_concept` dentro de esos periodos, puedo hacer la moda y asignar el `type_concept` más común.

In [ ]:
import scipy.stats as st

# I can iterate over idx_start and idx_end to get the
# periods
mode_values = []
for i in np.arange(len(idx_start)):
    df_tmp = df_rare.loc[idx_start[i] : idx_end[i]]
    print(f"{i=}")
    print(df_tmp[["person_id", "start_date", "end_date", "type_concept"]])
    mode = st.mode(df_tmp["type_concept"].values)
    print(f"mode is {mode}", "\n")
    mode_values.append(mode[0])

In [ ]:
# Add to dataframe
df_done["type_concept"] = mode_values
df_done

Y listo, ya tengo el dataframe con sólo los inicios y finales de los periodos, incluyendo el type_concept más común calculado usando la moda. Comprobamos que es igual que los resultados esperados:

In [ ]:
check_person = df_done["person_id"].values == df_result["person_id"].values
print(f"{'Person_id col is correct:':<28} {check_person.all()}")
check_start_date = df_done["start_date"].values == df_result["start_date"].values
print(f"{'start_date col is correct:':<28} {check_start_date.all()}")
check_end_date = df_done["end_date"].values == df_result["end_date"].values
print(f"{'end_date col is correct:':<28} {check_end_date.all()}")
check_type_concept = df_done["type_concept"].values == df_result["type_concept"].values
print(f"{'type_concept col is correct:'::<28} {check_type_concept.all()}")

### 2.2.2 Prueba con función recursive (NOT FINISHED)

ESTO ESTÁ AQUÍ PARA FUTURAS REFERENCIAS. EL CÓDIGO NO ESTÁ TERMINADO PORQUE EL MÉTODO POR ÍNDICES FUNCIONA LO SUFICIENTEMENTE BIEN Y NO HAY GARANTÍAS DE QUE ESTO LO MEJORE.

Parece que lo mejor (cof) va a ser repetir la estrategia anterior e iterar recursivamente. Así además nos aseguramos que podemos llevar la cuenta de los type_concept y quedarnos con el más representativo.)

In [ ]:
# def find_neighbors_index(df: pd.DataFrame,
#                          n_days: int) -> pd.Series:

#     # 1. Check that current and next patient are the same
#     idx_person = df.iloc[:, 0] == df.iloc[:, 0].shift(-1)
#     # 2. Check that current end_date and next start_date
#     # are closer than n_days
#     idx_interval = (
#         (df.iloc[:, 2] - df.iloc[:, 1].shift(-1)) <=
#         pd.Timedelta(n_days, unit='D')
#     )
#     # 4. If everything past is true, I can drop the row
#     return idx_person & idx_interval


# def remove_all_neighbors_recursive_v1(
#         df: pd.DataFrame,
#         n_days: int,
#         verbose: int = 0,
#         _counter: int = 0,
#         _counter_lim: int = 1000) -> pd.DataFrame:

#     # Get the rows
#     idx_to_remove = find_neighbors_index(df, n_days)
#     # Prepare next loop
#     idx_to_remove_sum = idx_to_remove.sum()
#     _counter += 1
#     # If there's still room to go, go
#     if (idx_to_remove_sum != 0) and (_counter < _counter_lim):
#         if verbose >= 1:
#             print(f"Iter {_counter} => {idx_to_remove_sum} rows removed.")
#         if verbose >= 2:
#             print(df[idx_to_remove].head(10))

#         # Modify end_dates
#         df.iloc[:,2] = np.where(idx_to_remove,
#                                 df.iloc[:,2].shift(-1),
#                                 df.iloc[:,2])
#         return remove_all_neighbors_recursive_v1(
#             df[idx_to_remove], verbose, _counter)
#     else:
#         return df

# n_days = 365
# df_rare = df_raw.sort_values(
#     ['person_id', 'start_date', 'end_date'],
#     ascending=[True, True, True])
# df_rare = remove_all_neighbors_recursive_v1(df_rare, n_days, verbose=2)
# df_done = df_rare.sort_index()
# df_done

In [ ]:
# n_days = 365
# df = df_raw.sort_values(
#     ['person_id', 'start_date', 'end_date'],
#     ascending=[True, True, True])

# # >>> Iter 1
# idx_to_remove = find_neighbors_index(df, n_days)
# print(f"Iter {1} => {idx_to_remove.sum()} rows removed.")
# print(df[idx_to_remove])
# # <<<

# # Record changes
# df['to_join'] = idx_to_remove
# df['new_end_date'] = np.where(df['to_join'],
#                               df.iloc[:, 2].shift(-1),
#                               df.iloc[:, 2])
# df

In [ ]:
# # >>> # Iter 2
# df.iloc[:,2] = np.where(idx_to_remove,
#                         df.iloc[:,2].shift(-1),
#                         df.iloc[:,2])
# idx_to_remove = find_neighbors_index(df, n_days)
# # <<<

# # Record changes
# df['to_join'] = idx_to_remove
# df['new_end_date'] = np.where(df['to_join'],
#                               df.iloc[:, 2].shift(-1),
#                               df.iloc[:, 2])
# df

In [ ]:
# # >>> # Iter 3
# df.iloc[:,2] = np.where(idx_to_remove,
#                         df.iloc[:,2].shift(-1),
#                         df.iloc[:,2])
# idx_to_remove = find_neighbors_index(df, n_days)
# # <<<

# # Record changes
# df['to_join'] = idx_to_remove
# df['new_end_date'] = np.where(df['to_join'],
#                               df.iloc[:, 2].shift(-1),
#                               df.iloc[:, 2])
# df

## 2.3 Todo junto

Ahora juntamos todo en una función, para poder medir el tiempo.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st


def find_person_index(df: pd.DataFrame) -> tuple[pd.Series]:
    """Finds all rows that are contained with the previous
    row, making sure they belong to the same person_id.

    Parameters
    ----------
    df : pd.DataFrame
        pandas Dataframe with at least three columns.
        Assumes first column is person_id, second column is
        start_date and third column is end_date

    Returns
    -------
    tuple[pd.Series]
        Tuple with three pandas Series with bools:
        - idx_person_first, True if first row of the person
        - idx_person_last, True if last row of the person
        - idx_person_only, True if only row of the person
        False otherwise.
    """

    # Create index for first, last or only person in dataset
    idx_person_first = (df.iloc[:, 0] == df.iloc[:, 0].shift(-1)) & (
        df.iloc[:, 0] != df.iloc[:, 0].shift(1)
    )
    idx_person_last = (df.iloc[:, 0] != df.iloc[:, 0].shift(-1)) & (
        df.iloc[:, 0] == df.iloc[:, 0].shift(1)
    )
    idx_person_only = (df.iloc[:, 0] != df.iloc[:, 0].shift(-1)) & (
        df.iloc[:, 0] != df.iloc[:, 0].shift(1)
    )
    return (idx_person_first, idx_person_last, idx_person_only)


def group_dates(df: pd.DataFrame, n_days: int) -> pd.DataFrame:
    """Groups rows of dates from the same person that are less
    than n_days apart, keeping only the first start_date and
    the last end_date, respectively.

    It will remove rows that are partially contained within
    the previous one.

    Parameters
    ----------
    df : pd.DataFrame
        pandas dataframe with at least four columns:
        ['person_id', 'start_date', 'end_date', 'type_concept'].
        Column names do not need to be the same but, the order
        must be the same as here.
        This allows its use for different tables with columns
        that have the same purpose but different names.
    verbose : int, optional
        Information output, by default 0
        - 0 No info
        - 1 Show number of iterations

    Returns
    -------
    pd.DataFrame
        Copy of input dataframe with grouped rows.
    """

    # == Preparation ==============================================
    # Sort so we know for sure the order is right
    df_rare = df.copy().sort_values(
        [df.columns[0], df.columns[1], df.columns[2]], ascending=[True, True, False]
    )
    # It is VERY important to reset the index to make sure we can
    # retrieve them realiably after sorting them.
    df_rare = df_rare.reset_index(drop=True)

    # == Index look-up ============================================
    (idx_person_first, idx_person_last, idx_person_only) = find_person_index(df_rare)
    # Create index if the break is too big and needs to be kept
    next_interval = df_rare.iloc[:, 1].shift(-1) - df_rare.iloc[:, 2]
    idx_interval = next_interval >= pd.Timedelta(n_days, unit="D")

    # == Retrieve relevant rows ===================================
    # -- start_date and person_id ---------------------------------
    # To retrieve the start_date we need the indexes of:
    # - single day periods (idx_person_only == True)
    # - first dates (idx_person_first == True)
    # - Rows just after period breaks, (idx_interval.index + 1)

    # Get the person condition indexes
    idx_start = df_rare.index[
        idx_person_only | idx_person_first | idx_interval.shift(1)
    ]

    # -- end_date -------------------------------------------------
    # Get the indexes
    idx_end = df_rare.index[idx_person_only | idx_person_last | idx_interval]

    # == Compute type_concept =====================================
    # Iterate over idx_start and idx_end to get the periods
    mode_values = []
    for i in np.arange(len(idx_start)):
        df_tmp = df_rare.loc[idx_start[i] : idx_end[i]]
        mode_values.append(st.mode(df_tmp.iloc[:, 3].values)[0])

    # == Build final dataframe ====================================
    # Create a copy (.loc) with the first two columns
    df_done = df_rare.loc[idx_start, [df.columns[0], df.columns[1]]]
    # Append values found to final dataframe
    df_done[df.columns[2]] = df_rare.loc[idx_end, [df.columns[2]]].values
    # Add to dataframe
    df_done[df.columns[3]] = mode_values

    return df_done

In [ ]:
# == Parametros ==
n_days = 365

# == Creación de datos ==
df_done = group_dates(df_raw, n_days)
df_done

volvemos a comprobar que sale bien

In [ ]:
check_person = df_done["person_id"].values == df_result["person_id"].values
print(f"{'Person_id col is correct:':<28} {check_person.all()}")
check_start_date = df_done["start_date"].values == df_result["start_date"].values
print(f"{'start_date col is correct:':<28} {check_start_date.all()}")
check_end_date = df_done["end_date"].values == df_result["end_date"].values
print(f"{'end_date col is correct:':<28} {check_end_date.all()}")
check_type_concept = df_done["type_concept"].values == df_result["type_concept"].values
print(f"{'type_concept col is correct:'::<28} {check_type_concept.all()}")

In [ ]:
%timeit -n 10 -r 10 group_dates(df_rare,n_days)

# 3. Prueba con datasets grandes
Vamos a comparar si el metodo de pyarrow sigue funcionando más rápido con datasets grandes.

Nos traemos la función para generar datasets

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
from pyarrow import parquet


def create_sample_df(
    n: int = 1000,
    n_dates: int = 50,
    first_date: str = "2020-01-01",
    last_date: str = "2023-01-01",
    mean_duration_days: int = 60,
    std_duration_days: int = 180,
) -> pd.DataFrame:

    # == Parameters ==
    np.random.seed(42)
    pd.options.mode.string_storage = "pyarrow"
    # Start date from which to start the dates
    first_date = pd.to_datetime(first_date)
    last_date = pd.to_datetime(last_date)
    max_days = (last_date - first_date).days

    # == Generate IDs randomly ==
    # -- Generate the Ids
    people = np.random.randint(10000000, 99999999 + 1, size=n)
    person_id = np.random.choice(people, n * n_dates)

    # == Generate random dates ==
    # Generate random integers for days and convert to timedelta
    random_days = np.random.randint(0, max_days, size=n * n_dates)
    # Create the columns
    observation_start_date = first_date + pd.to_timedelta(random_days, unit="D")
    # Generate a gaussian sample of dates
    random_days = np.random.normal(
        mean_duration_days, std_duration_days, size=n * n_dates
    )
    random_days = np.int32(random_days)
    observation_end_date = observation_start_date + pd.to_timedelta(
        random_days, unit="D"
    )
    # Correct end_dates
    # => If they are smaller than start_date, take start_date
    observation_end_date = np.where(
        observation_end_date < observation_start_date,
        observation_start_date,
        observation_end_date,
    )

    # == Generate the code ==
    period_type_concept_id = np.random.randint(1, 11, size=n * n_dates)

    # == Generate the dataframe ==
    df_raw = {
        "person_id": person_id,
        "observation_period_start_date": observation_start_date,
        "observation_period_end_date": observation_end_date,
        "period_type_concept_id": period_type_concept_id,
    }
    return pd.DataFrame(df_raw)

Nos traemos la función de pyarrow tal y como estaba el 12/09/2024

In [ ]:
import bps_to_omop.general as gen
import pyarrow as pa
import pyarrow.compute as pc
import numpy as np

# Añadimos el directorio superior al path para poder extraer
# las funciones de las carpetas ETL*
import sys
import os

# Add the parent directory of func_folder to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


def group_dates_original_pyarrow(table_done, n_days):
    # -- Thirdly, group up dates -----------------------------------------------
    # Agrupamos las fechas usando group_person_dates(). Básicamente calcula la
    # distancia temporal entre las filas adyacentes de cada persona, juntándolas
    # si es tan por debajo del límite marcado por n_days.
    # Agrupamos
    table_OBSERVATION_PERIOD = []

    person_list = pc.unique(table_done["person_id"])
    # Percentage points where you want to print progress
    for i, person in enumerate(person_list[:]):
        # --Group person
        table_person = group_person_dates(table_done, person, n_days)
        # Append table
        table_OBSERVATION_PERIOD.append(table_person)
    # Concatenate
    table_OBSERVATION_PERIOD = pa.concat_tables(table_OBSERVATION_PERIOD)
    return table_OBSERVATION_PERIOD


def group_person_dates(
    table_rare: pa.Table, person: str | int, n_days: int
) -> pa.Table:
    """Filters original table for a specific person and reduces
    the amount of date records grouping all records that are separated
    by n_days or less.

    Parameters
    ----------
    table_rare : pa.Table
        Table as prepared by 'prepare_table_raw_to_rare()'.
    person : str | int
        person id, can be an int (the usual) or a string.
    n_days : int
        number of maximum days between subsequent records.

    Returns
    -------
    pa.Table
        Table identical to table_rare but with less date records.
    """

    # Filter for the current person_id
    filt = pc.is_in(
        table_rare["person_id"], pa.array([person])  # pylint: disable=E1101
    )
    table_person = table_rare.filter(filt)
    # Retrieve corresponding dates
    start_dates = table_person["start_date"]
    end_dates = table_person["end_date"]
    # Group dates closer
    start_dates, end_dates, _ = group_observation_dates(
        start_dates, end_dates, n_days, verbose=False
    )
    # Create person
    person_id = pa_utils.create_uniform_int_array(len(start_dates), value=person)
    # Retrieve most common period type
    period_type_concept_id = pc.mode(  # pylint: disable=E1101
        table_person["period_type_concept_id"]
    )[0][0]
    period_type_concept_id = pa_utils.create_uniform_int_array(
        len(start_dates), value=period_type_concept_id
    )
    # return table
    return pa.Table.from_arrays(
        [person_id, start_dates, end_dates, period_type_concept_id],
        names=["person_id", "start_date", "end_date", "period_type_concept_id"],
    )


def group_observation_dates(
    start_dates: pa.Array, end_dates: pa.Array, n_days: int, verbose: bool = False
) -> tuple[pa.Array, pa.Array, None | pa.Table]:
    """Given a pair of 'start_dates' and 'end_dates', it will
        compute the days between each 'end_date' and the next
        'start_date' and remove dates that are smaller that a
        given number of days ('n_days').
    n_days = 365

    # Cargamos los datos
    df_raw = create_sample_df(n=100)
    df_raw.columns = ['person_id', 'start_date', 'end_date', 'type_concept']

    df_raw = df_raw.sort_values(
        ['person_id', 'start_date', 'end_date', 'type_concept'],
        ascending=[True, True, False, True])
    df_rare = df_raw.reset_index(drop=True).copy()

    print('\nshift:')
    %timeit -n 1 -r 1 group_dates(df_rare,n_days)

    df_rare = df_raw.reset_index(drop=True).copy()
    df_rare.columns = ['person_id', 'start_date', 'end_date', 'period_type_concept_id']
    table_rare = pa.Table.from_pandas(df_rare,preserve_index=False)
    print('\npyarrow:')
    %timeit -n 1 -r 1 group_dates_original_pyarrow(table_rare,n_days)
        The new dates will only contain start and end dates that have
        more than 'n_days' of difference between them.

        If dates contain nans/nulls, they will be ignored and grouped
        with the closest dates.

        Parameters
        ----------
        start_dates : pa.Array
            Array of start dates
        end_dates : pa.Array
            Array of end dates
        n_days : int
            _description_
        verbose : bool, optional
            _description_, by default False

        Returns
        -------
        tuple[pa.Array, pa.Array, None | pa.Table]
            Always return a 3-item tuple.
            First item is reduced start dates.
            Second item is reduced end dates.
            Third item is None if verbose=True,
            if verbose=False, is table with start_dates,
            end_dates and days between them. Usefull when
            verifying dates.

        Raises
        ------
        AssertionError
            The resulting starting dates should always come before
            their corresponding end dates. Return an AssertionError
            otherwise.
    """
    # Get an array of end_dates, taking away the last one
    # (last date cannot be compared to the next start date)
    from_dates = end_dates[:-1]
    # Get an array of start_dates, taking away the first one
    # (first date cannot be compared to the previous end date)
    to_dates = start_dates[1:]

    # -- Compute days between
    intervals = pc.days_between(from_dates, to_dates).to_numpy(  # pylint: disable=E1101
        zero_copy_only=False
    )
    # Create an inner table for the calculations if verbose
    inner_table = None
    if verbose:
        inner_table = pa.Table.from_arrays(
            [start_dates, end_dates, pa.array(np.append(intervals, np.nan))],
            names=["start", "end", "intervals"],
        )

    # Filter intervals under some assumption
    filt = intervals >= n_days
    # => When this filt is 'true', it means that for that index,
    # let's call it 'idx', between the end date of 'idx' and the start
    # date of 'idx+1' there more than 'n_days' days.
    # i.e.:
    # (start_date[idx+1] - end_date[idx]).days > n_days

    # if no interval is greater, take the first and last rows
    if np.nansum(filt) == 0:
        idx_end_dates = np.array([len(intervals)])
        # Sum 1 to get start dates
        idx_start_dates = np.array([0])

    # If some filters exist take those
    else:
        # Get indexes of end_dates
        idx_end_dates = filt.nonzero()[0]
        # Sum 1 to get corresponding start dates
        idx_start_dates = idx_end_dates + 1
        # Append last entry as last end_date
        idx_end_dates = np.append(idx_end_dates, len(intervals))
        # Append first entry as first start_date
        idx_start_dates = np.append(0, idx_start_dates)

    # if verbose:
    #     print(f'{idx_start_dates=}')
    #     print(f'{idx_end_dates=}')

    # Make sure all end values are after start values
    new_start = start_dates.take(idx_start_dates)
    new_end = end_dates.take(idx_end_dates)
    if pc.any(pc.less(new_end, new_start)).as_py():  # pylint: disable=E1101
        if verbose:
            print(f"{start_dates=}", f"{end_dates=}")
            print(f"{new_start=}", f"{new_end=}")
        raise AssertionError(
            "Some end dates happen before start dates. Try sorting the original data."
        )

    return (new_start, new_end, inner_table)

## 3.1. Sanity check
### DATA CREATION

Probamos primero que los resultados sean iguales con ambas funciones

In [ ]:
n_days = 365

# Cargamos los datos
df_raw = create_sample_df(
    n=100,
    n_dates=10,
)
df_raw.columns = ["person_id", "start_date", "end_date", "type_concept"]

df_raw = df_raw.sort_values(
    ["person_id", "start_date", "end_date", "type_concept"],
    ascending=[True, True, False, True],
)

Let's check how quick `remove_overlap()` is:

In [ ]:
%timeit -n 10 -r 10 remove_overlap(df_raw,0,False)

Quite quick!

In [ ]:
# Remove contained dates
df_raw = remove_overlap(df_raw, 0, False)

In [ ]:
df_raw[df_raw["person_id"] == 10271836]

In [ ]:
df_raw[df_raw["person_id"] == 23315092]

### PYARROW

In [ ]:
# == pyarrow method ==
df_rare = df_raw.copy()
df_rare.columns = ["person_id", "start_date", "end_date", "period_type_concept_id"]
table_rare = pa.Table.from_pandas(df_rare, preserve_index=False)
table_done = group_dates_original_pyarrow(table_rare, n_days)
df_done_pyarrow = table_done.to_pandas()
df_done_pyarrow = df_done_pyarrow.sort_values(
    ["person_id", "start_date", "end_date", "period_type_concept_id"],
    ascending=[True, True, False, True],
)

In [ ]:
df_done_pyarrow[df_done_pyarrow["person_id"] == 10271836]

In [ ]:
df_done_pyarrow[df_done_pyarrow["person_id"] == 23315092]

Pyarrow hace la primera persona (10271836), todas las fechas tienen menos de 365 días entre sí, así que se unen en una sola. Las siguientes son todas de una única fecha por persona hasta 23315092, que tiene dos. Esta también la hace bien.


### SHIFT

In [ ]:
# == Shift method ==
df_rare = df_raw.copy()
df_done_shift = group_dates(df_rare, n_days)

In [ ]:
df_done_shift[df_done_shift["person_id"] == 10271836]

In [ ]:
df_done_shift[df_done_shift["person_id"] == 23315092]

Ahora el método shift hace bien la primera persona (10271836) la que tiene dos periodos (23315092). El type_concept cambia del método pyarrow al shift, pero me fio más del shift en este momento.

## 3.2 Time measurement
Ahora probamos a medir el tiempo que tarda cada uno:

In [ ]:
n_days = 365

# Cargamos los datos
df_raw = create_sample_df(n=100)
df_raw.columns = ['person_id', 'start_date', 'end_date', 'type_concept']

df_raw = df_raw.sort_values(
    ['person_id', 'start_date', 'end_date', 'type_concept'],
    ascending=[True, True, False, True])
df_rare = df_raw.reset_index(drop=True).copy()

print('\nshift:')
%timeit -n 1 -r 1 group_dates(df_rare,n_days)

df_rare = df_raw.reset_index(drop=True).copy()
df_rare.columns = ['person_id', 'start_date', 'end_date', 'period_type_concept_id']
table_rare = pa.Table.from_pandas(df_rare,preserve_index=False)
print('\npyarrow:')
%timeit -n 1 -r 1 group_dates_original_pyarrow(table_rare,n_days)

Para n = 1000

    shift:
    375 ms ± 884 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)
    pyarrow:
    454 ms ± 664 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)

Para n = 10000

    shift:
    3.69 s ± 4.92 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
    pyarrow:
    23.6 s ± 565 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)

Para n = 30000

    shift:
    10.2 s ± 18.5 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
    pyarrow:
    3min 10s ± 5.35 s per loop (mean ± std. dev. of 2 runs, 2 loops each)
 

Ahora está bastante claro que el método shift funciona mucho más rápido si tenemos muchas personas. Al final en el método original estamos pegando tablas una encima de otra, lo cual resta mucho tiempo. Y esto teniendo en cuenta que el método shift está ordenando dentro de la propia función, cosa que en el de pyarrow dejamos fuera.

Quizá si se pudiera implementar con pyarrow un modo siguiendo el patrón de los shift, se podría conseguir algo mejor. Con todo, los 10 s con 30000 paciente y 50 fechas por paciente ya me parece un buen resultado.

## Prueba timestamp vs datetime

Nos hemos encontrado que el código remove_overlap va mucho más rápido si las fechas están en formato timestamp (pa.timestamp('us)) que si están en datetime (pa.date64()).

El problema está en que los datos finales en el proyecto `sarscov` no coinciden si se usa un método o el otro.

Probamos a lanzar el código aquí para comprobarlo.

In [ ]:
df_raw = create_sample_df(n=1000)
df_raw.columns = ["person_id", "start_date", "end_date", "type_concept"]

In [ ]:
n_days = 365
df_rare = df_raw.loc[:,['person_id','start_date','end_date','type_concept']]
table_raw = pa.Table.from_pandas(df_rare, preserve_index=False)
table_raw = table_raw.cast(
    pa.schema([
        ('person_id', pa.int64()),
        ('start_date', pa.timestamp('us')),
        ('end_date', pa.timestamp('us')),
        ('type_concept', pa.int64()),
    ])
)
df_rare = table_raw.to_pandas()
remove_overlap(df_rare,2).info()

%timeit -n 10 -r 10 group_dates_original_pyarrow(table_rare,n_days)

In [ ]:
n_days = 365
df_rare = df_raw.loc[:,['person_id','start_date','end_date','type_concept']]
table_raw = pa.Table.from_pandas(df_rare, preserve_index=False)
table_raw = table_raw.cast(
    pa.schema([
        ('person_id', pa.int64()),
        ('start_date', pa.date64()),
        ('end_date', pa.date64()),
        ('type_concept', pa.int64()),
    ])
)
df_rare = table_raw.to_pandas()
remove_overlap(df_rare,2).info()

%timeit -n 10 -r 10 group_dates_original_pyarrow(table_rare,n_days)

Ambos formatos funcionan bien, dejando el mismo número de filas.

Puede que el problema venga de que algunas fechas en los datos del proyecto vienen con hora. Por ejemplo, todas las de farmacia de dispensación. Si paso estos registros a date64 pierdo la información de la hora, por lo que el orden puede que sea distinto.

# 4. Nuevo método para calcular type_concept

Vamos a comparar el método actual de group_dates con hacer groupby

In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.stats as st

sys.path.append("../../")
from bps_to_omop.general import group_dates, find_person_index


def create_sample_data():
    nombre_columnas = ["person_id", "start_date", "end_date", "type_concept"]
    n_days = 365
    df_in = [
        # Una única fecha
        (1, "2020-01-01", "2020-02-01", 1),
        # Dos fechas que se juntan con type_concept iguales
        (2, "2020-01-01", "2020-02-01", 1),
        (2, "2020-03-01", "2020-04-01", 1),
        # Dos fechas que se juntan con type_concept distintos
        (3, "2020-01-01", "2020-02-01", 1),
        (3, "2020-03-01", "2020-04-01", 2),
        # tres fechas que se juntan
        (4, "2020-01-01", "2020-02-01", 1),
        (4, "2020-03-01", "2020-04-01", 1),
        (4, "2020-05-01", "2020-12-01", 2),
        # una persona con dos grupos distintos
        (5, "2020-01-01", "2020-02-01", 1),
        (5, "2020-03-01", "2020-04-01", 1),
        (5, "2020-05-01", "2020-12-01", 2),
        (5, "2022-01-01", "2022-02-01", 3),
        (5, "2022-03-01", "2022-04-01", 3),
        (5, "2022-05-01", "2022-12-01", 2),
    ]
    df_in = pd.DataFrame.from_records(df_in, columns=nombre_columnas).assign(
        start_date=lambda x: pd.to_datetime(x["start_date"]),
        end_date=lambda x: pd.to_datetime(x["end_date"]),
    )
    return df_in


def group_dates_v2(df: pd.DataFrame, n_days: int, verbose: int = 0) -> pd.DataFrame:
    # == Preparation ==============================================
    if verbose > 0:
        print("Grouping dates:")
        print("- Sorting and preparing data...")
    # Sort so we know for sure the order is right
    df_rare = df.copy().sort_values(
        [df.columns[0], df.columns[1], df.columns[2]], ascending=[True, True, False]
    )
    # It is VERY important to reset the index to make sure we can
    # retrieve them realiably after sorting them.
    df_rare = df_rare.reset_index(drop=True)

    # == Index look-up ============================================
    if verbose > 0:
        print("- Looking up indexes...")
    (idx_person_first, idx_person_last, idx_person_only) = find_person_index(df_rare)
    # Create index if the break is too big and needs to be kept
    next_interval = df_rare.iloc[:, 1].shift(-1) - df_rare.iloc[:, 2]
    idx_interval = next_interval >= pd.Timedelta(n_days, unit="D")

    # == Retrieve relevant rows ===================================
    if verbose > 0:
        print("- Retrieving rows...")
    # -- start_date and person_id ---------------------------------
    # To retrieve the start_date we need the indexes of:
    # - single day periods (idx_person_only == True)
    # - first dates (idx_person_first == True)
    # - Rows just after period breaks, (idx_interval.index + 1)

    # Get the person condition indexes
    idx_start = df_rare.index[
        idx_person_only | idx_person_first | idx_interval.shift(1)
    ]

    # -- end_date -------------------------------------------------
    # Get the indexes
    idx_end = df_rare.index[idx_person_only | idx_person_last | idx_interval]

    # == Compute type_concept =====================================
    if verbose > 0:
        print("- Computing type_concept...")
    # Iterate over idx_start and idx_end to get the periods
    mode_values = []
    for i in np.arange(len(idx_start)):
        df_tmp = df_rare.loc[idx_start[i] : idx_end[i]]
        mode_values.append(st.mode(df_tmp.iloc[:, 3].values)[0])

        if (verbose > 1) and ((i) % int(len(idx_start) / 4) == 0):
            print(f"  - ({(i+1)/len(idx_start)*100:.1f} %) {(i+1)}/{len(idx_start)}")
    if verbose > 1:
        print(f"  - (100.0 %) {len(idx_start)}/{len(idx_start)}")

    # == Build final dataframe ====================================
    if verbose > 0:
        print("- Closing up...")
    # Create a copy (.loc) with the first two columns
    df_done = df_rare.loc[idx_start, [df.columns[0], df.columns[1]]]
    # Append values found to final dataframe
    df_done[df.columns[2]] = df_rare.loc[idx_end, [df.columns[2]]].values
    # Add to dataframe
    df_done[df.columns[3]] = mode_values

    if verbose > 0:
        print("- Done!")
    return df_done

In [ ]:
verbose = 1
n_days = 365

df = create_sample_data()
df

**(!!)**

La idea aquí es que estamos buscando los índice y construyendo manualmente un dataframe con las fechas iniciales y finales.

NO podemos usar el truco del groupby para el type concept directamente, ya que no sabemos los intervalos finales.

Es decir, podemos agrupar por person_id, pero habría que agrupar también por las fechas, para poder sacar para cada persona y cada observation_period, cuál es el type_concept más frecuente.

In [ ]:
# == Preparation ==============================================
if verbose > 0:
    print("Grouping dates:")
    print("- Sorting and preparing data...")
# Sort so we know for sure the order is right
df_rare = df.copy().sort_values(
    [df.columns[0], df.columns[1], df.columns[2]], ascending=[True, True, False]
)
# It is VERY important to reset the index to make sure we can
# retrieve them realiably after sorting them.
df_rare = df_rare.reset_index(drop=True)

# == Index look-up ============================================
if verbose > 0:
    print("- Looking up indexes...")
(idx_person_first, idx_person_last, idx_person_only) = find_person_index(df_rare)
# Create index if the break is too big and needs to be kept
next_interval = df_rare.iloc[:, 1].shift(-1) - df_rare.iloc[:, 2]
idx_interval = next_interval >= pd.Timedelta(n_days, unit="D")

# == Retrieve relevant rows ===================================
if verbose > 0:
    print("- Retrieving rows...")
# -- start_date and person_id ---------------------------------
# To retrieve the start_date we need the indexes of:
# - single day periods (idx_person_only == True)
# - first dates (idx_person_first == True)
# - Rows just after period breaks, (idx_interval.index + 1)

# Get the person condition indexes
idx_start = df_rare.index[idx_person_only | idx_person_first | idx_interval.shift(1)]

# -- end_date -------------------------------------------------
# Get the indexes
idx_end = df_rare.index[idx_person_only | idx_person_last | idx_interval]

# == Compute type_concept =====================================
if verbose > 0:
    print("- Computing type_concept...")
# Iterate over idx_start and idx_end to get the periods
mode_values = []
for i in np.arange(len(idx_start)):
    df_tmp = df_rare.loc[idx_start[i] : idx_end[i]]
    mode_values.append(st.mode(df_tmp.iloc[:, 3].values)[0])

    if (verbose > 1) and ((i) % int(len(idx_start) / 4) == 0):
        print(f"  - ({(i+1)/len(idx_start)*100:.1f} %) {(i+1)}/{len(idx_start)}")
if verbose > 1:
    print(f"  - (100.0 %) {len(idx_start)}/{len(idx_start)}")

# == Build final dataframe ====================================
if verbose > 0:
    print("- Closing up...")
# Create a copy (.loc) with the first two columns
df_done = df_rare.loc[idx_start, [df.columns[0], df.columns[1]]]
# Append values found to final dataframe
df_done[df.columns[2]] = df_rare.loc[idx_end, [df.columns[2]]].values
# # Add to dataframe
df_done[df.columns[3]] = mode_values

if verbose > 0:
    print("- Done!")
df_done

# A. Unfinished testing code

In [ ]:
import pandas as pd
import numpy as np
import time
from typing import List, Tuple


def assign_groups_masking(
    indices: np.ndarray, starts: np.ndarray, ends: np.ndarray
) -> np.ndarray:
    """Group assignment using boolean masking"""
    group_ids = np.zeros(len(indices), dtype=int)
    for group_num, (start, end) in enumerate(zip(starts, ends), 1):
        mask = (indices >= start) & (indices <= end)
        group_ids[mask] = group_num
    return group_ids


def assign_groups_searchsorted(
    indices: np.ndarray, starts: np.ndarray, ends: np.ndarray
) -> np.ndarray:
    """Group assignment using searchsorted"""
    boundaries = np.sort(np.concatenate([starts, ends + 1]))
    return np.searchsorted(boundaries, indices, side="right") // 2


def generate_test_case(n_rows: int, n_groups: int) -> Tuple[np.ndarray, np.ndarray]:
    """Generate test data with given size and number of groups"""
    # Create roughly equal-sized groups
    group_size = n_rows // n_groups
    starts = np.arange(0, n_rows, group_size)
    ends = starts + group_size - 1
    ends[-1] = n_rows - 1  # Adjust last group
    return starts, ends


def run_benchmark():
    # Test configurations
    row_sizes = [10_000, 100_000, 1_000_000]
    group_configs = [
        ("Few Large Groups", lambda x: max(5, x // 1_000_000)),
        ("Medium Groups", lambda x: max(50, x // 100_000)),
        ("Many Small Groups", lambda x: max(500, x // 10_000)),
    ]

    results = []

    for n_rows in row_sizes:
        indices = np.arange(n_rows)

        for group_desc, group_func in group_configs:
            n_groups = group_func(n_rows)
            starts, ends = generate_test_case(n_rows, n_groups)

            # Warm-up run
            _ = assign_groups_masking(indices, starts, ends)
            _ = assign_groups_searchsorted(indices, starts, ends)

            # Timing masking approach
            start_time = time.perf_counter()
            for _ in range(5):  # Multiple runs for more stable results
                _ = assign_groups_masking(indices, starts, ends)
            masking_time = (time.perf_counter() - start_time) / 5

            # Timing searchsorted approach
            start_time = time.perf_counter()
            for _ in range(5):
                _ = assign_groups_searchsorted(indices, starts, ends)
            searchsorted_time = (time.perf_counter() - start_time) / 5

            results.append(
                {
                    "Rows": n_rows,
                    "Groups": n_groups,
                    "Configuration": group_desc,
                    "Masking Time": masking_time,
                    "Searchsorted Time": searchsorted_time,
                }
            )

    return pd.DataFrame(results)


# Run benchmark
results_df = run_benchmark()

# Print detailed results
print("\nDetailed Benchmark Results:")
print("=" * 80)
for _, row in results_df.iterrows():
    print(f"\nConfiguration: {row['Configuration']}")
    print(f"Data Size: {row['Rows']:,} rows, {row['Groups']:,} groups")
    print(f"Masking Time: {row['Masking Time']*1000:.2f}ms")
    print(f"Searchsorted Time: {row['Searchsorted Time']*1000:.2f}ms")
    speedup = row["Masking Time"] / row["Searchsorted Time"]
    faster_method = "searchsorted" if speedup > 1 else "masking"
    print(
        f"Winner: {faster_method} ({abs(speedup):,.2f}x {'faster' if speedup > 1 else 'slower'})"
    )

# Calculate and print summary statistics
print("\nSummary Statistics:")
print("=" * 80)
for config in results_df["Configuration"].unique():
    config_results = results_df[results_df["Configuration"] == config]
    print(f"\n{config}:")
    avg_speedup = (
        config_results["Masking Time"] / config_results["Searchsorted Time"]
    ).mean()
    print(f"Average speedup using searchsorted: {avg_speedup:.2f}x")

# Validation of correctness
print("\nValidating correctness of implementations...")
test_indices = np.arange(1000)
test_starts = np.array([0, 200, 400, 600, 800])
test_ends = np.array([199, 399, 599, 799, 999])

masking_results = assign_groups_masking(test_indices, test_starts, test_ends)
searchsorted_results = assign_groups_searchsorted(test_indices, test_starts, test_ends)

if np.array_equal(masking_results, searchsorted_results):
    print("✓ Both implementations produce identical results")
else:
    print("⚠ WARNING: Implementations produce different results!")